# PINN Burgers TF1 GPU

*Last edited 2023-05-05*

TensorFlow v1 PINN, adapted from the original Raissi PINN: https://github.com/maziarraissi/PINNs/blob/master/appendix/continuous_time_inference%20(Burgers)/Burgers.py

Running on Nvidia Geforce GTX1050 3GB 6.1

Import TensofFlow v1.15:

In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
    sys.exit([1])
print(tf.__version__)

GPU found
1.15.0


2023-02-08 12:45:25.199944: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2023-02-08 12:45:25.222130: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2599990000 Hz
2023-02-08 12:45:25.223099: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4aff540 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-08 12:45:25.223120: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-02-08 12:45:25.224585: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-08 12:45:25.336701: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x49e3980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-08 12:45:25.336717: I tensorflow/compil

Disable TensorFlow logging output, if desired'':

In [2]:
# import logging, os
# logging.disable(logging.WARNING)
# logging.getLogger('tensorflow').disabled = True
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.get_logger().setLevel(logging.ERROR)
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [4]:
import scipy.io
from scipy.interpolate import griddata

- [pyDOE](https://pythonhosted.org/pyDOE/randomized.html#latin-hypercube)
(includes the Latin-Hypercube)

In [5]:
from pyDOE import lhs

In [6]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
from numba import cuda
import numpy as np
import time

In [8]:
np.set_printoptions(linewidth=132, precision=6)

Try to make the result reproducible:

In [9]:
np.random.seed(1234)
tf.compat.v1.set_random_seed(1234)

## Data

In [10]:
DATA_BURGERS = 'burgers_shock.mat'

In [11]:
data  = scipy.io.loadmat(DATA_BURGERS)
Exact = np.real(data['usol']).T

In [12]:
Exact.shape

(100, 256)

In [13]:
t = data['t'].flatten()[:, None]
x = data['x'].flatten()[:, None]

In [14]:
t.shape

(100, 1)

NT = 100

In [15]:
x.shape

(256, 1)

NX = 256

In [16]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Sep 20 16:38:22 2017',
 '__version__': '1.0',
 '__globals__': [],
 'x': array([[-1.      ],
        [-0.992157],
        [-0.984314],
        [-0.976471],
        [-0.968627],
        [-0.960784],
        [-0.952941],
        [-0.945098],
        [-0.937255],
        [-0.929412],
        [-0.921569],
        [-0.913725],
        [-0.905882],
        [-0.898039],
        [-0.890196],
        [-0.882353],
        [-0.87451 ],
        [-0.866667],
        [-0.858824],
        [-0.85098 ],
        [-0.843137],
        [-0.835294],
        [-0.827451],
        [-0.819608],
        [-0.811765],
        [-0.803922],
        [-0.796078],
        [-0.788235],
        [-0.780392],
        [-0.772549],
        [-0.764706],
        [-0.756863],
        [-0.74902 ],
        [-0.741176],
        [-0.733333],
        [-0.72549 ],
        [-0.717647],
        [-0.709804],
        [-0.701961],
        [-0.694118],
        [-0.686275]

In [17]:
Exact[0:5,5:10].T

array([[0.122888, 0.119137, 0.115608, 0.112283, 0.109144],
       [0.147302, 0.142819, 0.138601, 0.134624, 0.13087 ],
       [0.171626, 0.166422, 0.161523, 0.156904, 0.15254 ],
       [0.195845, 0.189933, 0.184364, 0.17911 , 0.174145],
       [0.219946, 0.213338, 0.207111, 0.201233, 0.195676]])

## Plotting module

In [18]:
# mpl.use('pgf')


def figsize(scale, nplots=1):
    fig_width_pt = 390.0  # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0 / 72.27  # Convert pt to inch

    # Aesthetic ratio (you could change this)
    golden_mean = (np.sqrt(5.0) - 1.0) / 2.0

    fig_width = fig_width_pt * inches_per_pt * scale  # width in inches
    fig_height = nplots * fig_width * golden_mean  # height in inches
    fig_size = [fig_width, fig_height]
    return fig_size


##setup matplotlib to use latex for output
pgf_with_latex = {
    ##change this if using xetex or latex
    "pgf.texsystem": "pdflatex",
    ##use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    ##blank entries should cause plots to inherit fonts from the document
    "font.serif": [],
    "font.sans-serif": [],
    "font.monospace": [],
    ##LaTeX default is 10pt font.
    "axes.labelsize": 10,
    "font.size": 10,
    ##Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    ##default fig size of 0.9 textwidth
    "figure.figsize": figsize(1.0),
    "pgf.preamble": [
        ##use utf8 fonts becasue your computer can handle it :)
        #r"\usepackage[utf8x]{inputenc}",
        ##plots will be generated using this preamble
        r"\usepackage[T1]{fontenc}"]}

plt.rcParams.update(pgf_with_latex)

import matplotlib.pyplot as plt


# I make my own savefig functions
def savefig(filename, crop=True):
    if crop == True:
        # plt.savefig('{}.pgf'.format(filename),
        #             bbox_inches='tight',
        #             pad_inches=0)
        plt.savefig('{}.pdf'.format(filename),
                    bbox_inches='tight',
                    pad_inches=0)
        plt.savefig('{}.eps'.format(filename),
                    bbox_inches='tight',
                    pad_inches=0)
    else:
        # plt.savefig('{}.pgf'.format(filename))
        plt.savefig('{}.pdf'.format(filename))
        plt.savefig('{}.eps'.format(filename))

## PINN code

In [19]:
class PhysicsInformedNN:

    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        self.lb = lb
        self.ub = ub
        self.x_u = X_u[:, 0:1]
        self.t_u = X_u[:, 1:2]
        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]
        self.u = u
        self.layers = layers
        self.nu = nu

        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)

        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        self.x_u_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.t_u.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32,
                                     shape=[None, self.t_f.shape[1]])
        self.u_pred = self.net_u(self.x_u_tf, self.t_u_tf)
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(
            self.loss,
            method='L-BFGS-B',
            options={
                'maxiter': 50000,
                'maxfun': 50000,
                'maxcor': 50,
                'maxls': 50,
                'ftol': 1.0 * np.finfo(float).eps
            })
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = self.xavier_init(size=[layers[l], layers[l + 1]])
            b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                            dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim],
                                               stddev=xavier_stddev),
                           dtype=tf.float32)

    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for l in range(0, num_layers - 2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

    def net_u(self, x, t):
        u = self.neural_net(tf.concat([x, t], 1), self.weights, self.biases)
        return u

    def net_f(self, x, t):
        u = self.net_u(x, t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = u_t + u * u_x - self.nu * u_xx
        return f

    #def callback(self, loss):
    #    print('Loss:', loss)

    def train(self):
        tf_dict = {
            self.x_u_tf: self.x_u,
            self.t_u_tf: self.t_u,
            self.u_tf: self.u,
            self.x_f_tf: self.x_f,
            self.t_f_tf: self.t_f
        }
        self.optimizer.minimize(self.sess,
                                feed_dict=tf_dict,
                                fetches=[self.loss],
                                #loss_callback=self.callback
                               )

    def predict(self, X_star):
        u_star = self.sess.run(self.u_pred, {
            self.x_u_tf: X_star[:, 0:1],
            self.t_u_tf: X_star[:, 1:2]
        })
        f_star = self.sess.run(self.f_pred, {
            self.x_f_tf: X_star[:, 0:1],
            self.t_f_tf: X_star[:, 1:2]
        })
        return u_star, f_star

## Main

In [20]:
N_u = 100  # no_of_collocation_points (100)
N_f = 10000  # no_of_interior_points (10000)
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
nu = 0.01 / np.pi

data = scipy.io.loadmat(DATA_BURGERS)

t = data['t'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = np.real(data['usol']).T  # Exact
X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]  # Exact

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

xx1 = np.hstack((X[0:1, :].T, T[0:1, :].T))
xx2 = np.hstack((X[:, 0:1], T[:, 0:1]))
xx3 = np.hstack((X[:, -1:], T[:, -1:]))
X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub - lb) * lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]

# Exact
uu1 = Exact[0:1, :].T
uu2 = Exact[:, 0:1]
uu3 = Exact[:, -1:]
u_train = np.vstack([uu1, uu2, uu3])
u_train = u_train[idx, :]  # Exact

## Model

In [21]:
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)




Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1




2023-02-08 12:45:32.003220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.442
pciBusID: 0000:01:00.0
2023-02-08 12:45:32.003273: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-02-08 12:45:32.003286: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-02-08 12:45:32.003295: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2023-02-08 12:45:32.003305: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2023-02-08 12:45:32.003314: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10.0
2023-02-08 12:45:32.003323: I tensorflow/stream_exec

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2023-02-08 12:45:33.925534: I tensorflow/core/common_runtime/placer.cc:54] truncated_normal/TruncatedNormal: (TruncatedNormal): /job:localhost/replica:0/task:0/device:GPU:0
2023-02-08 12:45:33.925561: I tensorflow/core/common_runtime/placer.cc:54] truncated_normal/mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
2023-02-08 12:45:33.925568: I tensorflow/core/common_runtime/placer.cc:54] truncated_normal: (Add): /job:localhost/replica:0/task:0/device:GPU:0
2023-02-08 12:45:33.925572: I tensorflow/core/common_runtime/placer.cc:54] Variable: (VariableV2): /job:localhost/replica:0/task:0/device:GPU:0
2023-02-08 12:45:33.925577: I tensorflow/core/common_runtime/placer.cc:54] Variable/Assign: (Assign): /job:localhost/replica:0/task:0/device:GPU:0
2023-02-08 12:45:33.925580: I tensorflow/core/common_runtime/placer.cc:54] Variable/read: (Identity): /job:localhost/replica:0/task:0/device:GPU:0
2023-02-08 12:45:33.925585: I tensorflow/core/common_runtime/placer.cc:54] Variable_1: (Variabl

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1
truncated_normal/TruncatedNormal: (TruncatedNormal): /job:localhost/replica:0/task:0/device:GPU:0
truncated_normal/mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
truncated_normal: (Add): /job:localhost/replica:0/task:0/device:GPU:0
Variable: (VariableV2): /job:localhost/replica:0/task:0/device:GPU:0
Variable/Assign: (Assign): /job:localhost/replica:0/task:0/device:GPU:0
Variable/read: (Identity): /job:localhost/replica:0/task:0/device:GPU:0
Variable_1: (VariableV2): /job:localhost/replica:0/task:0/device:GPU:0
Variable_1/Assign: (Assign): /job:localhost/replica:0/task:0/device:GPU:0
Variable_1/read: (Identity): /job:localhost/replica:0/task:0/device:GPU:0
truncated_normal_1/T

## Compute intensive part

In [22]:
%%time
history = model.train()

2023-02-08 12:45:35.352077: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0


INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.000002
  Number of iterations: 5684
  Number of functions evaluations: 6265
CPU times: user 3min 23s, sys: 10.6 s, total: 3min 34s
Wall time: 2min 13s


## Predict

In [23]:
%%time
u_pred, f_pred = model.predict(X_star)

CPU times: user 240 ms, sys: 8.39 ms, total: 248 ms
Wall time: 245 ms


In [24]:
# tf.compat.v1.disable_eager_execution()

In [25]:
# tf.config.threading.set_intra_op_parallelism_threads(1)
# tf.config.threading.set_inter_op_parallelism_threads(1)

In [26]:
from tensorflow.python.client import device_lib

In [27]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16696663612701310838
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9671753290856482403
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13381083137045319206
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2769289216
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7419511909672712110
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


2023-02-08 12:47:50.495753: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.442
pciBusID: 0000:01:00.0
2023-02-08 12:47:50.495867: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-02-08 12:47:50.495919: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-02-08 12:47:50.495963: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2023-02-08 12:47:50.496006: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2023-02-08 12:47:50.496047: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10.0
2023-02-08 12:47:50.496088: I tensorflow/stream_exec

In [28]:
tf.test.is_gpu_available()

2023-02-08 12:47:51.005369: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.442
pciBusID: 0000:01:00.0
2023-02-08 12:47:51.005522: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-02-08 12:47:51.005607: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-02-08 12:47:51.005671: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2023-02-08 12:47:51.005735: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2023-02-08 12:47:51.005797: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10.0
2023-02-08 12:47:51.005887: I tensorflow/stream_exec

True

In [29]:
tf.test.gpu_device_name()

2023-02-08 12:47:51.526077: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.442
pciBusID: 0000:01:00.0
2023-02-08 12:47:51.526184: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-02-08 12:47:51.526234: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-02-08 12:47:51.526276: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2023-02-08 12:47:51.526316: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2023-02-08 12:47:51.526356: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10.0
2023-02-08 12:47:51.526397: I tensorflow/stream_exec

'/device:GPU:0'

In [30]:
tf.debugging.set_log_device_placement(True)

In [31]:
%%time
with tf.device('/GPU:0'):
    u_pred, f_pred = model.predict(X_star)

CPU times: user 28.9 ms, sys: 252 µs, total: 29.2 ms
Wall time: 17 ms


## Error u

In [32]:
error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
print('Error u:', error_u)

Error u: 0.0018419930448150014


## Save data

In [33]:
model.save('burgerstf1230208.h5')

AttributeError: 'PhysicsInformedNN' object has no attribute 'save'

In [ ]:
np.save('data_history.npy', history)
np.save('data_u_pred.npy', u_pred)
np.save('data_f_pred.npy', f_pred)

---

## Load data

In [ ]:
history = np.load('data_history.npy', allow_pickle=True)
u_pred = np.load('data_u_pred.npy', allow_pickle=True)
f_pred = np.load('data_f_pred.npy', allow_pickle=True)

## Error

In [ ]:
U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
Error = np.abs(Exact - U_pred)

## Plotting

## $u(t,x)$ graph

In [ ]:
WIDTH = 1.0  # width in inches.
NPLOTS = 0.8  # height in inches.
fig = plt.figure(figsize=figsize(WIDTH, NPLOTS))
ax = fig.add_subplot(111)  # (nrows=1, ncols=1, index=1)

h = ax.imshow(U_pred.T,
              extent=[t.min(), t.max(), x.min(),
                      x.max()],
              aspect='auto')

# right side bar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

# add vertical lines
line = np.linspace(x.min(), x.max(), 2)[:, None]
ax.plot(t[25] * np.ones((2, 1)), line, '--', label="$t = 0.25$")
ax.plot(t[75] * np.ones((2, 1)), line, '--', label="$t = 0.75$")

# plot the axis marks
ax.plot(X_u_train[:, 1],
        X_u_train[:, 0],
        'rx',
        label='Data (%d points)' % (u_train.shape[0]),
        markersize=4,
        clip_on=False)
# message inside the figure, on the right side
#ax.legend()

# title and labels
font = {'size': 'large', 'weight': 'bold', 'color': 'blue'}
# bottom vertical alignment for more space
ax.set_title('$u(t,x)$', verticalalignment='bottom', **font)
ax.set_xlabel('$t$', **font)
ax.set_ylabel('$x$', **font)

ax.legend(bbox_to_anchor=(.95, -.25), ncol=4)

plt.show()

## $u(t,x)$ slices

In [ ]:
font = {'size': 'large', 'weight': 'bold', 'color': 'blue'}
gs = gridspec.GridSpec(1, 2, wspace=.5)

ax = plt.subplot(gs[0, 0])
ax.plot(x, Exact[25, :], '-', linewidth = 2, label='Exact')
ax.plot(x, U_pred[25, :], '--', linewidth = 2, label='Prediction')
ax.set_title('$t=0.25$', verticalalignment='bottom', **font)
ax.set_xlabel('$x$', **font)
ax.set_ylabel('$u(t,x)$', **font)
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])

ax = plt.subplot(gs[0, 1])
ax.plot(x, Exact[75, :], '-', linewidth = 2, label='Exact')
ax.plot(x, U_pred[75, :], '--', linewidth = 2, label='Prediction')
ax.set_title('$t=0.75$', verticalalignment='bottom', **font)
ax.set_xlabel('$x$', **font)
ax.set_ylabel('$u(t,x)$', **font)
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])

ax.legend(bbox_to_anchor=(0.3, -0.2), ncol=2)

plt.show()

## Release gpu memory

In [ ]:
cuda.get_current_device().reset()